# K-Means Clustering

[Clustering](http://en.wikipedia.org/wiki/Cluster_analysis), or cluster analysis, is an exploratory data analysis tool for grouping a set of objects in such a way that objects in the same group (called a **cluster**) are more similar (in some sense) to each other than to those in other groups (clusters).

Cluster analysis itself is not one specific algorithm, but the general task to be done. It can be achieved by various algorithms that differ significantly in their notion of what constitutes a cluster and how to efficiently find them. Popular notions of clusters include groups with small distances among the cluster members, dense areas of the data space, intervals or particular statistical distributions.

One of the most common clustering techniques is [K-Means clustering](http://en.wikipedia.org/wiki/K-means_clustering) which is simple to understand but which produces powerful results.

## Introducing K-Means

K-Means is an **unsupervised learning** algorithm. It finds clusters in data based on the data attributes alone without labels.

K Means searches for cluster centers, which are the mean of the points within them, such that every point is closest to the cluster center it is assigned to.

Let's look at how K-Means operates on some simple clusters we will generate using the `sklearn` random blob generator. To emphasize that this is unsupervised, we won't plot the colors of the clusters.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# use seaborn plotting defaults
# If this causes an error, you can comment it out.
import seaborn as sns
sns.set()

from sklearn.datasets.samples_generator import make_blobs

In [ ]:
X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], s=50);

By eye, it is relatively easy to pick out the four clusters. If you were to perform an exhaustive search for the different segmentations of the data, however, the search space would be exponential in the number of points. Fortunately, there is a well-known *Expectation Maximization (EM)* procedure which `scikit-learn` implements, so that K-Means can be solved relatively quickly.

In [ ]:
from sklearn.cluster import KMeans
KM = KMeans(4)  # 4 clusters
KM.fit(X)
y_kmeans = KM.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='rainbow');

The algorithm identifies the four clusters of points in a manner very similar to what we would do by eye!

### Interactive view of the algorithm

We can see how the K-Means algorithm works with the help of a nice function from [Jake Vanderplas](https://github.com/jakevdp):

In [ ]:
def plot_kmeans_interactive(min_clusters=1, max_clusters=6):
    from IPython.html.widgets import interact
    from sklearn.metrics.pairwise import euclidean_distances
    from sklearn.datasets.samples_generator import make_blobs
    
    import warnings
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')

        X, y = make_blobs(n_samples=300, centers=4,
                          random_state=0, cluster_std=0.60)

        def _kmeans_step(frame=0, n_clusters=4):
            rng = np.random.RandomState(2)
            labels = np.zeros(X.shape[0])
            centers = rng.randn(n_clusters, 2)

            nsteps = frame // 3

            for i in range(nsteps + 1):
                old_centers = centers
                if i < nsteps or frame % 3 > 0:
                    dist = euclidean_distances(X, centers)
                    labels = dist.argmin(1)

                if i < nsteps or frame % 3 > 1:
                    centers = np.array([X[labels == j].mean(0)
                                        for j in range(n_clusters)])
                    nans = np.isnan(centers)
                    centers[nans] = old_centers[nans]


            # plot the data and cluster centers
            plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='rainbow',
                        vmin=0, vmax=n_clusters - 1);
            plt.scatter(old_centers[:, 0], old_centers[:, 1], marker='o',
                        c=np.arange(n_clusters),
                        s=200, cmap='rainbow')
            plt.scatter(old_centers[:, 0], old_centers[:, 1], marker='o',
                        c='black', s=50)

            # plot new centers if third frame
            if frame % 3 == 2:
                for i in range(n_clusters):
                    plt.annotate('', centers[i], old_centers[i], 
                                 arrowprops=dict(arrowstyle='->', linewidth=1))
                plt.scatter(centers[:, 0], centers[:, 1], marker='o',
                            c=np.arange(n_clusters),
                            s=200, cmap='rainbow')
                plt.scatter(centers[:, 0], centers[:, 1], marker='o',
                            c='black', s=50)

            plt.xlim(-4, 4)
            plt.ylim(-2, 10)

            if frame % 3 == 1:
                plt.text(3.8, 9.5, "1. Reassign points to nearest centroid",
                         ha='right', va='top', size=14)
            elif frame % 3 == 2:
                plt.text(3.8, 9.5, "2. Update centroids to cluster means",
                         ha='right', va='top', size=14)

    
    return interact(_kmeans_step, frame=[0, 50],
                    n_clusters=[min_clusters, max_clusters])

In [ ]:
import numpy as np
plot_kmeans_interactive();

### K-Means Caveats

The convergence of this algorithm is not guaranteed. For that reason, `scikit-learn` by default uses a large number of random initializations and finds the best results.

Also, the number of clusters must be set beforehand... there are other clustering algorithms for which this requirement may be lifted.

## Clustering the Iris data

The [Iris flower data](http://en.wikipedia.org/wiki/Iris_flower_data_set) is one of the classics. It has been used for many years in statistical analysis and machine learning settings.

The data set consists of 50 samples from each of three species of Iris (*Iris setosa*, *Iris virginica* and *Iris versicolor*). Four features were measured from each sample: the length and the width of the [sepals](http://en.wikipedia.org/wiki/Sepal) and [petals](http://en.wikipedia.org/wiki/Petal) in centimeters.

<img src="http://5047-presscdn.pagely.netdna-cdn.com/wp-content/uploads/2015/04/iris_petal_sepal.png">

### Loading the Iris Data with Scikit-Learn

The Iris data is one of the datasets included with `scikit-learn`. The data consist of the following:

- Features in the Iris dataset:

  1. sepal length in cm
  2. sepal width in cm
  3. petal length in cm
  4. petal width in cm


- Target classes to predict:

  1. Iris Setosa
  2. Iris Versicolour
  3. Iris Virginica
  
`scikit-learn` embeds a copy of the iris data file along with a helper function to load it into numpy arrays:

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

### Let's explore the data

In [ ]:
iris.keys()

In [ ]:
iris.data.shape

In [ ]:
n_samples, n_features = iris.data.shape
n_samples, n_features

In [ ]:
iris.target.shape

In [ ]:
print(iris.target)

In [ ]:
print(iris.target_names)

For this dataset, we know the labels (the species). We can do a simple scatter plot for two of the attributes:

In [ ]:
X = iris.data[:, :2]
Y = iris.target

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

plt.scatter(X[:, 0], X[:, 1], c=Y, cmap='rainbow')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max);

### Use Seaborn for a nice visualization

Seaborn also includes the Iris dataset. We can load it then use the `pairplot` function to get a better view of the data. Note that this creates a `pandas` DataFrame.

In [ ]:
df = sns.load_dataset("iris")
type(df)

In [ ]:
df.head()

This seems like a cleaner way to work with the data!

Now let's do that `pairplot`.

In [ ]:
sns.pairplot(df, hue="species");

### A note about `pandas`

Note that we could have created the same `pandas` DataFrame from the `scikit-learn` version of the dataset:

In [ ]:
import pandas as pd
df2 = pd.DataFrame(iris.data, columns=iris.feature_names)
df2.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
df2['species']=iris.target_names[iris.target]

In [ ]:
df2.head()

For what follows, it will be convenient to create a column for 'cluster' and drop the 'species' column:

In [ ]:
if 'species' in df2.columns:
    df2=df2.drop('species', 1)
df2['cluster']=iris.target

In [ ]:
df2.head()

## Clustering

We can look at the Seaborn `pairplot` and see that the data seems to cluster well for some attributes and not for others.

Suppose we did NOT have the iris dataset labels. Could we use cluster analysis to discover natural clusters? Would they be related to species?

We start by specifying the number of clusters.

In [ ]:
n_clusters=3

In [ ]:
X, y = iris.data, iris.target

k_means = KMeans(n_clusters, random_state=2)
k_means.fit(X)
y_pred = k_means.predict(X)


In [ ]:
y_pred

In [ ]:
data=np.array([y,y_pred]).T

In [ ]:
compare=pd.DataFrame(data)
compare.columns = ['actual', 'predicted']

In [ ]:
compare['same'] = (compare['actual'] == compare['predicted'])

In [ ]:
compare.head()

In [ ]:
compare.same.sum()

Look at the Seaborn `pairplot` again:

In [ ]:
df2['cluster'] = y_pred
sns.pairplot(df2, hue="cluster");

### Explore changing "K"

Try changing the value of `n_clusters` and running the cell that follows:

In [ ]:
n_clusters = 4

In [ ]:
X, y = iris.data, iris.target

k_means = KMeans(n_clusters, random_state=2)
k_means.fit(X)
y_pred = k_means.predict(X)

df2['cluster'] = y_pred
sns.pairplot(df2, hue="cluster");

## Another possibility: use PCA (principal component analysis) to reduce the dimensionality of the iris data for visualization

[PCA](http://en.wikipedia.org/wiki/Principal_component_analysis) is a common method for reducing the dimensionality of a dataset. It is very closely related to the SVD (singular value decomposition) we used when we looked at the gene expression data.

PCA finds the linear combinations of the original variables that explains the overall variability. Quite often, the first few principal components explain enough of the variability to allow for further analysis using just those components.

We'll use `scikit-learn` to do PCA on the iris data then use the first two components for plotting the results of clustering.

In [ ]:
from sklearn.decomposition import PCA

X, y = iris.data, iris.target
pca = PCA(n_components=2)
pca.fit(X)
X_reduced = pca.transform(X)

print
print("Reduced dataset shape:", X_reduced.shape)

plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, cmap='rainbow')

print
print("Meaning of the 2 components:")
for component in pca.components_:
    print(" + ".join("%.3f x %s" % (value, name)
                     for value, name in zip(component,
                                            iris.feature_names)))

Now we can plot the clusters against these same principal components:

In [ ]:
n_clusters = 3

In [ ]:
X, y = iris.data, iris.target

k_means = KMeans(n_clusters, random_state=2)
k_means.fit(X)
y_pred = k_means.predict(X)

plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y_pred, cmap='rainbow');

## Evaluating clustering results

There are many ways to evaluate the choice of "K" in K-Means clustering. We won't spend time on this here, but Wikipedia has an excellent [explanation](http://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set) if you're interested in learning more.